In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('STRESS.csv')
df.head()

,snoring rate,respiration rate,body temperature,limb movement,blood oxygen,retina eye movement,sleeping rate,heart rate,stress level
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3.0
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3.0
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1.0
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3.0
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0.0


In [3]:
df=df.dropna()

In [4]:
df['stress level'].value_counts()

0.0    2523
4.0    2408
2.0    2322
3.0    2287
1.0    2251
Name: stress level, dtype: int64

In [5]:
df.columns

Index(['snoring rate', 'respiration rate', 'body temperature', 'limb movement',
       'blood oxygen', 'retina eye movement', 'sleeping rate', 'heart rate',
       'stress level'],
      dtype='object')

In [6]:
df.head()

,snoring rate,respiration rate,body temperature,limb movement,blood oxygen,retina eye movement,sleeping rate,heart rate,stress level
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3.0
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3.0
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1.0
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3.0
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0.0


In [7]:
x1 = df.drop(labels='stress level', axis=1).values
y1 = df.loc[:,'stress level'].values

In [8]:
# Data Preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features = scaler.fit_transform(x1)

In [9]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros =RandomOverSampler(random_state=42)
x,y=ros.fit_resample(x1,y1)
print("OUR DATASET COUNT         : ", Counter(y1))
print("OVER SAMPLING DATA COUNT  : ", Counter(y))

OUR DATASET COUNT         :  Counter({0.0: 2523, 4.0: 2408, 2.0: 2322, 3.0: 2287, 1.0: 2251})
OVER SAMPLING DATA COUNT  :  Counter({3.0: 2523, 1.0: 2523, 0.0: 2523, 2.0: 2523, 4.0: 2523})


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42, stratify=y)
print("NUMBER OF TRAIN DATASET    : ", len(x_train))
print("NUMBER OF TEST DATASET      : ", len(x_test))
print("TOTAL NUMBER OF DATASET    : ", len(x_train)+len(x_test))

NUMBER OF TRAIN DATASET    :  10092
NUMBER OF TEST DATASET      :  2523
TOTAL NUMBER OF DATASET    :  12615


In [11]:
print("NUMBER OF TRAIN DATASET    : ", len(y_train))
print("NUMBER OF TEST DATASET      : ", len(y_test))
print("TOTAL NUMBER OF DATASET    : ", len(y_train)+len(y_test))

NUMBER OF TRAIN DATASET    :  10092
NUMBER OF TEST DATASET      :  2523
TOTAL NUMBER OF DATASET    :  12615


In [12]:
# Convert features and labels to torch tensors
features = torch.tensor(x, dtype=torch.float32)
labels = torch.tensor(y, dtype=torch.long)


In [13]:
# Step 2: Split the dataset into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42
)


In [14]:
# Step 3: Create DataLoader for batch processing
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:

# Step 4: Define the neural network model
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [16]:
# Step 5: Create the model and set hyperparameters
input_size = 8
hidden_size = 128
num_classes = 5
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Initialize lists to store loss and accuracy for each epoch
epoch_losses = []
epoch_accuracies = []

In [17]:
from sklearn.metrics import accuracy_score

# Step 6: Training the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    num_batches = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1

    average_loss = running_loss / num_batches
    
    # Calculate accuracy at the end of each epoch
    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        test_predictions = []
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_predictions.extend(predicted.cpu().numpy())

        # Calculate accuracy
        accuracy = accuracy_score(test_labels, test_predictions)
        epoch_accuracies.append(accuracy)
        epoch_losses.append(average_loss)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy * 100:.2f}%, Loss: {average_loss:.4f}')

    model.train()  # Switch back to training mode

print('Training finished.')

Epoch [1/20], Accuracy: 99.60%, Loss: 0.8408
Epoch [2/20], Accuracy: 100.00%, Loss: 0.1258
Epoch [3/20], Accuracy: 100.00%, Loss: 0.0521
Epoch [4/20], Accuracy: 100.00%, Loss: 0.0272
Epoch [5/20], Accuracy: 100.00%, Loss: 0.0157
Epoch [6/20], Accuracy: 100.00%, Loss: 0.0100
Epoch [7/20], Accuracy: 100.00%, Loss: 0.0068
Epoch [8/20], Accuracy: 100.00%, Loss: 0.0048
Epoch [9/20], Accuracy: 100.00%, Loss: 0.0035
Epoch [10/20], Accuracy: 100.00%, Loss: 0.0027
Epoch [11/20], Accuracy: 100.00%, Loss: 0.0020
Epoch [12/20], Accuracy: 100.00%, Loss: 0.0016
Epoch [13/20], Accuracy: 100.00%, Loss: 0.0012
Epoch [14/20], Accuracy: 100.00%, Loss: 0.0010
Epoch [15/20], Accuracy: 100.00%, Loss: 0.0008
Epoch [16/20], Accuracy: 100.00%, Loss: 0.0006
Epoch [17/20], Accuracy: 100.00%, Loss: 0.0005
Epoch [18/20], Accuracy: 100.00%, Loss: 0.0004
Epoch [19/20], Accuracy: 100.00%, Loss: 0.0003
Epoch [20/20], Accuracy: 100.00%, Loss: 0.0003
Training finished.
